Library Installation

In [ ]:
import sys
!{sys.executable} -m pip install googletrans
!{sys.executable} -m pip install genanki

Initialization

In [69]:
from googletrans import Translator
import genanki

async def translate_text(text: str, src_lang: str, dest_lang: str) -> str:
    async with Translator() as translator:
        result = await translator.translate(text, src=src_lang, dest=dest_lang)
        return result.text

async def lang_of_text(text: str) -> str:
    async with Translator() as translator:
        result = await translator.detect(text)
        return result.lang
    
anki_model = genanki.Model(
    1215581359,
    "Leet Model 1215581359",
    fields=[{"name": "Question"}, {"name": "Answer"}],
    templates=[{"name": "Card 1", "qfmt": "{{Question}}", "afmt": '{{FrontSide}}<hr id="answer">{{Answer}}'}]
)

Set File

In [70]:
meta = (input("What file are you basing the cards off of?"), input("What is the original language? (use language code)"))

Load File into Memory

In [71]:
file_text = ""
with open(meta[0], encoding="utf-8") as file: file_text = file.read()

Create Deck

In [ ]:
import random

random.seed(meta[0])
deck = genanki.Deck(random.randrange(1 << 30, 1 << 31), meta[0].split(".")[0])

def removeSymbols(text: str) -> str:
    for sym in "!.,\"\\/?¿¡:;«»@#$%^&*()_=+|[]{}1234567890": text = text.replace(sym, "")
    return text

file_text = removeSymbols(file_text)
added_words = []

for word in file_text.split():
    if word in added_words: continue
    if await lang_of_text(word) != meta[1]: continue
    translated = await translate_text(word, meta[1], "en")
    added_words.append(word)
    deck.add_note(genanki.Note(model=anki_model, fields=[f"<h1>{word}</h1>", f"<h1>{translated}</h1>"]))
    print(f"Words added: {len(added_words)}. Added {word}.")

Save Deck

In [73]:
genanki.Package(deck).write_to_file(meta[0].split(".")[0] + ".apkg")